# (1) Train predicator (classifier) for AFA 
The AFA problem contains the training of a classifier that is able to predict well based on any acquired subset of features. 
In this notebook we train and save such a classifier 

In [1]:
%load_ext autoreload
%autoreload 2

## Define paths

Paths for data

In [2]:
from afa.configurations.utils_ts import specify_default_paths_ts
# which dataset to work on 
dataset_name   = "miiv_test"

# name for of missingness scenario 
miss_scenario  = 'MCAR_1'

# automatically specify some path locations (change paths manually if needed) 
paths = specify_default_paths_ts(dataset_name = dataset_name , miss_scenario = miss_scenario) 

Paths for model

In [10]:
# name for predictor 
predictor_model_name  = 'TCN'

# new (where to save the model) 
predictor_model_dir = paths['data_dir'] + 'predictor_models' + '/' + predictor_model_name + '/'

## Load dataset with missingness 
At first, we want to load the dataset 

Includes loading: 
- superfeature mapping
- problem
- afa_problem 
- missingness_model

In [4]:
from afa.data_modelling.datasets.data_loader.data_loader_ts import DataLoader_ts

2023-08-07 16:02:06.585841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 16:02:06.701156: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-07 16:02:06.701174: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-07 16:02:09.841973: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
data_loader = DataLoader_ts(     data_file                  = paths['data_file'],
                                 temporal_data_file         = paths['temporal_data_file'],
                                 superfeature_mapping_file  = paths['superfeature_mapping_file'],
                                 problem_file               = paths['problem_file'],
                                 afa_problem_files          = paths['afa_problem_files'], 
                                 miss_model_files           = paths['miss_model_files'], 
                                 folds_file                 = paths['folds_file'] )
dataset = data_loader.load() 

Padding sequences: 100%|██████████| 100/100 [00:00<00:00, 1524.17it/s]


## Create afa dataset with missingness 
To train an AFA predictor, we need to induce artificial missingness (such that the predictor is robust to the missingness pattern). 
Here, we use a simple model to create missingness randomly (MCAR).

In [6]:
from afa.afa_datasets.utils_ts import create_MCAR_afa_dataset_ts

In [7]:
afa_dataset = create_MCAR_afa_dataset_ts( dataset, MCAR_ratio = 0.9, n_samples = 5)

Change base_model to stationary_constant
Change base_model to stationary_constant
Set weight for constant stationary model ...
Set weight for constant stationary model ...


## Train Predictor 

### Initialize predictor 

In [8]:
from afa.afa_models.afa_predictors.afa_predictor_ts import AFAPredictor_ts

In [14]:
predictor_model_params = {
    'name' : predictor_model_name, 
    'directory': predictor_model_dir,
    'base_model_params' : {   'model_type': 'ImputeThenRegress',
                              'imputer_params' : 
                                       {   
                                      'model_type': 'simple_imputer',
                                      'mode' : 'imputation'
                                      },
                              'predictor_params' : 
                                       {   
                                      'model_type': 'ann',
                                      'mode' : 'classification',
                                      'units':              1,
                                      'layers': 1,
                                      'learning_rate': 0.01,
                                      'batch_size' :128,
                                      'epochs': 100 
                                      }
                          }
    }

predictor_model_params = {
    'name' : predictor_model_name, 
    'directory': predictor_model_dir,
    'base_model_params' : {   
        'model_type': 'TCN',  
        'seed': 54,                     # seed for reproduceability
        'lr': 0.01,                     # initial learning rate for lr-scheduler
        'time excluded': True,          # for logging purposes, if time is included as a feature
        'num_inputs': 48,                # The number of features that are used as input
        'num_layers': 2,                # The number of temporal conv. layers
        'channels_per_layer': 8,        # The number of channels per temp. conv. layer
        'kernel_size': 3,               # Num time steps for kernel window
        'max_epochs': 100,              # Max. epochs for training
        'batch_size': 2000,             # Number of samples per batch
        'dropout': 0.1,                 # Dropout for all temp. conv. layers
        'patience': 40,                 # Num epochs as patience for early stopping, default 50.
        'weight_decay': 0,              # Weight decay for L2 regularization of Adam optmizer
        'wandb_logger': False,           # If weights and biases logger should be used
        #'wandb_project_name': 'Synthetic_2 classifier (AFA)'    # project name if wandb_logger is used
    }
}

In [15]:
afa_predictor = AFAPredictor_ts(    name            = predictor_model_params['name'], 
                                    model_params    = predictor_model_params, 
                                    directory       = predictor_model_params['directory'])   

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Instantiating the TCN_Lightning model...
No previously trained model found.
Conv1d layer initialized with Kaiming Uniform.
MLP part input size:  8
Value recognized as pad value: -1
TCN_lightning(
  (loss): MaskedBCEWithLogitsLoss()
  (model): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1dWithWN(48, 8, kernel_size=(3,), stride=(1,), padding=(2,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.1, inplace=False)
        (conv2): Conv1dWithWN(8, 8, kernel_size=(3,), stride=(1,), padding=(2,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.1, inplace=False)
        (net): Sequential(
          (0): Conv1dWithWN(48, 8, kernel_size=(3,), stride=(1,), padding=(2,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): Conv1dWithWN(8, 8, kernel_size=(3,), stride=(1,), padding=(2,))
          (5): Chomp1d()
          (6): Re

In [16]:
afa_predictor.fit(afa_dataset, 
                  fold = 0, 
                  train_split = 'train', 
                  valid_split = 'val', 
                  fit_again = True)

Convert superR to R: 100%|██████████| 2/2 [00:00<00:00, 3240.10it/s]


Global seed set to 54
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                    | Params
--------------------------------------------------
0 | loss  | MaskedBCEWithLogitsLoss | 0     
1 | model | TemporalConvNet         | 2.2 K 
2 | fc    | Sequential              | 9     
--------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Batch.shape:  torch.Size([26, 120, 49])


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Monitored metric val_loss = nan is not finite. Previous best value was inf. Signaling Trainer to stop.


Careful, using val_dataloader for final metrics. Please use test_dataloader if you have one.
Applying sigmoid to y_pred!
(1680,) (1680,) (1680,) (1680,)
y_true: (1680,), y_logits: (1680,), y_score: (1680,), y_pred: (1680,)


ValueError: Input y_true contains NaN.

In [25]:
label_pred = afa_predictor.predict( afa_dataset, fold = 0, split = 'val', n_samples = 1 )

Convert superR to R: 100%|██████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2932.05it/s]


### Write report

In [26]:
explanation_file = afa_predictor.directory + 'model_report.md'  # +  'reports/' + 'model_report' 
afa_predictor.explain(file= explanation_file, format = 'markdown')